In [2]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.8/269.8 kB 3.7 MB/s eta 0:00:00 MB/s eta 0:00:01


In [19]:
from langchain.document_loaders import PyPDFLoader
 
loader = PyPDFLoader("./samsunginsurance.pdf")
data = loader.load_and_split()

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[
                                               "\n\n", "\n", "(?<=\. )", " "], length_function=len)
docs = text_splitter.split_documents(data)

In [21]:
import os
# bring your own OPENAI API KEY
os.environ["OPENAI_API_KEY"] = "<openapi_api_key>"

from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model_name='gpt-3.5-turbo')

In [22]:
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch

mongodb_connectionstring="<connection string>"
mongodb_dbname="test"
mongodb_collname="langchain_insurance"
mongodb_indexname="langchain_insurance"

client = MongoClient(mongodb_connectionstring)
collection = client[mongodb_dbname][mongodb_collname]

# Insert the documents in MongoDB Atlas with their embedding
docsearch = MongoDBAtlasVectorSearch.from_documents(
    docs, embeddings, collection=collection, index_name=mongodb_indexname
)

## index build

mongodb_indexname="langchain_insurance"

```bash
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "type": "knnVector",
        "dimensions": 1536,
        "similarity": "cosine"
      }
    }
  }
}
```

In [45]:
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

client = MongoClient(mongodb_connectionstring)
collection = client[mongodb_dbname][mongodb_collname]

vectorStore = MongoDBAtlasVectorSearch(
    collection, OpenAIEmbeddings(model_name='gpt-3.5-turbo'), index_name=mongodb_indexname
)

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

chain = RetrievalQAWithSourcesChain.from_chain_type(
   llm=llm,
   chain_type = 'stuff',
   retriever=vectorStore.as_retriever(search_kwargs={'k':3}),
   return_source_documents=True
)


In [35]:
def ChatBot(question):
    answer = chain(question)
    print('Q. ' + answer['question'])
    print('---------------------------------')
    print('A. ' + answer['answer'])
    print('source: ' + answer['sources'])
    print('original content: ' + answer['source_documents'][0].page_content)

In [36]:
ChatBot("자동차 사고시 대처요령에 대해서 자세히 설명해주세요")

Q. 자동차 사고시 대처요령에 대해서 자세히 설명해주세요
---------------------------------
A. 사고 시 대처 요령은 다음과 같습니다:
1. 사고현장에서는 즉시 정차하고 사고를 확인해야 합니다.
2. 부상자가 있는 경우, 부상 상태를 확인하고 응급 조치 후 구급차량을 호출해야 합니다.
3. 정황증거를 확보하기 위해 사고현장을 사진촬영하고 사고장소 위치를 도로상에 표시해야 합니다. 또한 목격자의 정보를 확보해야 합니다.
4. 사고차량을 안전한 장소로 이동시켜야 합니다.
5. 필요한 긴급조치를 한 후에는 경찰서에 신고해야 합니다.

source: ./samsunginsurance.pdf
original content: 121. 자동차 사고 시 대처 요령
⑴  사고현장 사고처리  
사고현장에서 꼭 필요로 하는 적절한 대처요령을 사전에  
확인하시기 바랍니다.Ⅴ . 가입자 유의사항 : 자동차사고 시 보상처리 절차
1 즉시 정차
 · 어떤 사고라도 일단 정차한 후 사고 확인  
 · 피해차량이라도 일단 정차 후 사고 확인
2 부상자 구호
 · 부상 상태 확인  
 ·  부상이 심할 경우 응급 조치 후 구급차량으로 후송
3 정황증거 확보
 ·  사고현장을 여러 각도에서 사진촬영(핸드폰, 카메라 등)
 ·  사고장소 위치 등을 도로상 표시(스프레이, 페인트 등)
 ·  목격자 확보(인적사항, 연락처)
4 안전한 장소로 사고차량 이동
 ·  정황증거확보 후 교통상황이 안전에 위협이 되는 경우 
위험방지를 위해 가까운 길 가장자리나 안전한 장소로 
차량을 이동하고 엔진을 완전히 정지시킨다.
5 필요한 긴급조치 후 경찰서 신고
 ·  차량만 파손된 것이 분명하고 도로에서의 위험방지와 
원활한 소통을 위하여 필요한 조치를 했을 때에는 반드시 
신고할 필요는 없습니다.
 *  사고 현장에서 전문지식 없이 옳고 그름을 임의로 판단해서는 
당사자 간에 감정의 골만 깊어지는 경우가 많으니 법률전문가의 
도움을받거나 보험회사에 위임하는 것이 좋습니다.


In [37]:
ChatBot("긴급출동 서비스에 대해서 설명해줘")

Q. 긴급출동 서비스에 대해서 설명해줘
---------------------------------
A. 긴급출동 서비스는 외제차 및 전기차 운반비용 지원 특별약관에 가입한 경우에 제공됩니다. 이 서비스는 사고처리시 비용을 지원하고 긴급출동 서비스를 제공합니다. 

source: ./samsunginsurance.pdf
original content: [45-2] 긴급견인서비스 확대 추가특별약관(전기차)  ……… 177
   [45-3] 차량 진단 Door to Door 서비스 추가특별약관  ……… 177Ⅴ. 긴급출동 서비스
[46] 보험료 분할납입 특별약관  …………………………………… 179
[47] 신용카드 이용 보험료 납입 특별약관  ……………………… 180
[48] 보험료 자동납입 특별약관  …………………………………… 180
[49] 보험계약 자동갱신 특별약관  ………………………………… 182Ⅵ. 보험료 납입


In [46]:
ChatBot("사고시 보험금 지급절차에 대해서 알려줘")

Q. 사고시 보험금 지급절차에 대해서 알려줘
---------------------------------
A. 보험금의 지급절차에 대해서는 다음과 같습니다:
1. 피보험자는 사고가 발생한 때 보험회사에 대하여 보험금의 지급을 청구할 수 있습니다.
2. 피보험자가 보험금의 지급을 청구할 때에는 보험금청구서, 손해액을 증명하는 서류, 그 밖에 보험회사가 필요하다고 인정하는 서류 또는 증거를 제출하여야 합니다.
3. 회사는 보험금 청구에 관한 서류를 받은 때부터 10일 이내에 결정하여 지급합니다.
4. 회사가 보험금 지급사유의 조사 및 확인을 위하여 지급 기일을 초과할 것으로 예상되는 경우에는 구체적 사유와 지급 예정일을 피보험자에게 문서로 통지합니다.
5. 회사가 지급 기일 내에 보험금을 지급하지 않았을 때에는 그 다음날부터 지급일까지의 기간에 대하여 보험개발원이 공시한 보험계약대출이율에 의한 이자를 보험금에 더하여 지급합니다. 단, 피보험자 또는 보험계약자에게 책임 있는 사유로 지급이 지연된 경우에는 해당 기간에 대한 이자는 지급하지 않습니다.

source: ./samsunginsurance.pdf
original content: ①  피보험자는 사고가 발생한 때 보험회사에 대하여 보험금의 지급을 
청구할 수 있습니다.
②  피보험자가 보험금의 지급을 청구할 때에는 보험금청구서, 손해액
을 증명하는 서류, 그 밖에 보험회사가 꼭 필요하다고 인정하는 서류 
또는 증거를 제출하여야 합니다.
③  회사는 위 ‘②’에 정한 보험금 청구에 관한 서류를 받은 때부터 10일 
이내에 결정하여 지급합니다.
④  회사가 보험금 지급사유의 조사 및 확인을 위하여 위 ‘③’의 지급 기
일 초과가 명백히 예상되는 경우에는 구체적 사유와 지급 예정일을 
피보험자에게 문서로 통지하여 드립니다.
⑤  회사는 위 ‘③’ 또는 ‘④’의 규정에서 정한 지급 기일 내에 보험금을 
지급하지 아니하였을 때에는 그 다음날부터 지급일까지의 기간에 
대하여 보험개발원이 공시한 보험계약대출이율에 의한 이

## prompt

In [39]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), use "SOURCES" in capital letters regardless of the number of sources.

If you don't know the answer, just make it up by any means.

----------------
{summaries}

You MUST answer in Korean and in Markdown format:"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [40]:
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

client = MongoClient(mongodb_connectionstring)
collection = client[mongodb_dbname][mongodb_collname]

vectorStore = MongoDBAtlasVectorSearch(
    collection, OpenAIEmbeddings(), index_name=mongodb_indexname
)

chain_type_kwargs = {"prompt": prompt}
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

chain = RetrievalQAWithSourcesChain.from_chain_type(
   llm=llm,
   chain_type = 'stuff',
   retriever=vectorStore.as_retriever(search_kwargs={'k':3}),
   return_source_documents=True,
   chain_type_kwargs=chain_type_kwargs
)

In [41]:
ChatBot("자동차 사고시 대처요령에 대해서 자세히 설명해주세요")

Q. 자동차 사고시 대처요령에 대해서 자세히 설명해주세요
---------------------------------
A. 자동차 사고 시 대처 요령은 다음과 같습니다:

1. 즉시 정차: 어떤 사고라도 일단 정차한 후 사고를 확인해야 합니다. 피해차량이라도 일단 정차 후 사고를 확인해야 합니다.
2. 부상자 구호: 부상 상태를 확인하고, 부상이 심할 경우 응급 조치 후 구급차량으로 후송해야 합니다.
3. 정황증거 확보: 사고현장을 여러 각도에서 사진 촬영하고, 사고장소 위치 등을 도로상에 표시해야 합니다. 또한, 목격자의 인적사항과 연락처를 확보해야 합니다.
4. 안전한 장소로 사고차량 이동: 정황증거 확보 후 교통상황이 안전에 위협이 되는 경우, 가까운 길 가장자리나 안전한 장소로 차량을 이동하고 엔진을 완전히 정지시켜야 합니다.
5. 필요한 긴급조치 후 경찰서 신고: 차량만 파손된 것이 분명하고 도로에서의 위험방지와 원활한 소통을 위해 필요한 조치를 했을 때에는 반드시 신고할 필요는 없습니다. 하지만, 전문지식 없이 옳고 그름을 판단하는 것은 좋지 않으므로 법률전문가의 도움이나 보험회사에 위임하는 것이 좋습니다.

이러한 대처 요령은 사고 현장에서의 대응을 원활하게 하고, 보험처리 절차를 수월하게 진행하기 위한 것입니다. 사고 시에는 안전을 최우선으로 생각하고, 필요한 조치를 취해야 합니다.

[SOURCES](./samsunginsurance.pdf)
source: 
original content: 121. 자동차 사고 시 대처 요령
⑴  사고현장 사고처리  
사고현장에서 꼭 필요로 하는 적절한 대처요령을 사전에  
확인하시기 바랍니다.Ⅴ . 가입자 유의사항 : 자동차사고 시 보상처리 절차
1 즉시 정차
 · 어떤 사고라도 일단 정차한 후 사고 확인  
 · 피해차량이라도 일단 정차 후 사고 확인
2 부상자 구호
 · 부상 상태 확인  
 ·  부상이 심할 경우 응급 조치 후 구급차량으로 후송
3 정황증거 확보
 ·  사고현장을 여러 각도에서 사진촬영(

In [42]:
ChatBot("긴급출동 서비스에 대해서 설명해줘")

Q. 긴급출동 서비스에 대해서 설명해줘
---------------------------------
A. 긴급출동 서비스는 삼성화재에서 제공하는 서비스 중 하나입니다. 이 서비스는 차량이 고장이나 사고로 인해 긴급한 도움이 필요한 상황에서 도움을 받을 수 있도록 도와줍니다. 긴급출동 서비스에는 긴급견인서비스와 긴급출동진단서비스가 포함되어 있습니다.

긴급견인서비스는 차량이 고장이나 사고로 인해 움직일 수 없는 상황에서 차량을 견인해주는 서비스입니다. 이 서비스를 통해 차량을 안전하게 정비소로 이동시킬 수 있습니다.

긴급출동진단서비스는 차량이 고장이나 사고로 인해 도움이 필요한 경우, 전문 기사가 차량에 진단을 실시하여 문제를 파악하고 적절한 조치를 취해줍니다. 이를 통해 차량의 고장을 빠르게 해결할 수 있습니다.

긴급출동 서비스는 삼성화재의 특별약관에 따라 제공되며, 자세한 내용은 해당 특별약관을 참고하시기 바랍니다. [45-2]와 [45-3]의 내용을 확인하시면 더 자세한 정보를 얻을 수 있습니다. (SOURCES)
source: 
original content: [45-2] 긴급견인서비스 확대 추가특별약관(전기차)  ……… 177
   [45-3] 차량 진단 Door to Door 서비스 추가특별약관  ……… 177Ⅴ. 긴급출동 서비스
[46] 보험료 분할납입 특별약관  …………………………………… 179
[47] 신용카드 이용 보험료 납입 특별약관  ……………………… 180
[48] 보험료 자동납입 특별약관  …………………………………… 180
[49] 보험계약 자동갱신 특별약관  ………………………………… 182Ⅵ. 보험료 납입


In [44]:
ChatBot("사고시 보험금 지급절차에 대해서 알려줘")

Q. 사고시 보험금 지급절차에 대해서 알려줘
---------------------------------
A. 보험금 지급 절차는 다음과 같습니다:

1. 피보험자는 사고 발생 시 보험회사에 보험금 지급을 청구할 수 있습니다.
2. 보험금 지급을 청구할 때에는 보험금 청구서, 손해액을 증명하는 서류, 그 외 보험회사가 필요하다고 인정하는 서류나 증거를 제출해야 합니다.
3. 보험회사는 위의 서류를 받은 후 10일 이내에 결정하여 보험금을 지급합니다.
4. 만약 보험회사가 보험금 지급 사유를 조사하고 확인하기 위해 10일 이내에 지급 기일을 초과할 것으로 예상되는 경우, 구체적인 사유와 지급 예정일을 피보험자에게 문서로 통지합니다.
5. 보험회사는 위의 규정에서 정한 지급 기일 내에 보험금을 지급하지 않았을 경우, 다음 날부터 지급일까지의 기간에 보험개발원이 공시한 보험계약 대출 이율에 의한 이자를 보험금에 추가로 지급합니다. 단, 피보험자나 보험계약자에게 책임 있는 사유로 지급이 지연된 경우에는 해당 기간에 대한 이자는 추가로 지급하지 않습니다.

이와 관련된 자세한 내용은 해당 보험 계약서를 참조하시기 바랍니다. [^1^]

[SOURCES]
[^1^]: ./samsunginsurance.pdf
source: 
original content: ①  피보험자는 사고가 발생한 때 보험회사에 대하여 보험금의 지급을 
청구할 수 있습니다.
②  피보험자가 보험금의 지급을 청구할 때에는 보험금청구서, 손해액
을 증명하는 서류, 그 밖에 보험회사가 꼭 필요하다고 인정하는 서류 
또는 증거를 제출하여야 합니다.
③  회사는 위 ‘②’에 정한 보험금 청구에 관한 서류를 받은 때부터 10일 
이내에 결정하여 지급합니다.
④  회사가 보험금 지급사유의 조사 및 확인을 위하여 위 ‘③’의 지급 기
일 초과가 명백히 예상되는 경우에는 구체적 사유와 지급 예정일을 
피보험자에게 문서로 통지하여 드립니다.
⑤  회사는 위 ‘③’ 또는 ‘④’의 규정에서 정한 지급 기일 내에 보험금을 